## Module 8: Homework

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch

from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

#### Random Seed

In [3]:
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

#### Data Loading

In [4]:
data_transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [5]:
train_data = datasets.ImageFolder(
    root="data/train",
    transform=data_transform,
    target_transform=None
)

val_data = datasets.ImageFolder(
    root="data/test",
    transform=data_transform
)

print(f"Train:\n{train_data}\nVal:\n{val_data}")

Train:
Dataset ImageFolder
    Number of datapoints: 800
    Root location: data/train
    StandardTransform
Transform: Compose(
               Resize(size=(200, 200), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
Val:
Dataset ImageFolder
    Number of datapoints: 201
    Root location: data/test
    StandardTransform
Transform: Compose(
               Resize(size=(200, 200), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )


In [6]:
train_data.classes

['curly', 'straight']

In [7]:
img, label = train_data[0][0], train_data[0][1]
img.shape

torch.Size([3, 200, 200])

In [8]:
train_dl = DataLoader(
    dataset=train_data,
    batch_size=20,
    shuffle=True
)

val_dl = DataLoader(
    dataset=val_data,
    batch_size=20,
    shuffle=False
)

train_dl, val_dl

(<torch.utils.data.dataloader.DataLoader at 0x7ab14d66c740>,
 <torch.utils.data.dataloader.DataLoader at 0x7ab14e185760>)

#### Model Creation

In [9]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(3, 32, 3),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32*99*99, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        x = self.conv_block(x)
        output = self.classifier(x)
        return output

In [10]:
model = NeuralNetwork()
model

NeuralNetwork(
  (conv_block): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=313632, out_features=64, bias=True)
    (2): ReLU()
    (3): Linear(in_features=64, out_features=1, bias=True)
  )
)

> Q1: Which loss function you will use?

In [11]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)
criterion = nn.BCEWithLogitsLoss() # for binary classification

> Q2: The total number of parameters using?

In [12]:
# Option 2: Manual counting
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 20073473


#### Model Training

In [13]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [15]:
num_epochs = 10
history1 = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_dl:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_data)
    epoch_acc = correct_train / total_train
    history1['loss'].append(epoch_loss)
    history1['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in val_dl:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(val_data)
    val_epoch_acc = correct_val / total_val
    history1['val_loss'].append(val_epoch_loss)
    history1['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6705, Acc: 0.6000, Val Loss: 0.6094, Val Acc: 0.6468
Epoch 2/10, Loss: 0.5593, Acc: 0.7037, Val Loss: 0.5892, Val Acc: 0.6915
Epoch 3/10, Loss: 0.5207, Acc: 0.7312, Val Loss: 0.5878, Val Acc: 0.6517
Epoch 4/10, Loss: 0.4628, Acc: 0.7688, Val Loss: 0.6816, Val Acc: 0.6368
Epoch 5/10, Loss: 0.4259, Acc: 0.8013, Val Loss: 0.6590, Val Acc: 0.6716
Epoch 6/10, Loss: 0.3505, Acc: 0.8438, Val Loss: 0.8655, Val Acc: 0.6468
Epoch 7/10, Loss: 0.3027, Acc: 0.8725, Val Loss: 0.6974, Val Acc: 0.6816
Epoch 8/10, Loss: 0.2523, Acc: 0.8900, Val Loss: 0.7179, Val Acc: 0.7114
Epoch 9/10, Loss: 0.2289, Acc: 0.9163, Val Loss: 0.7017, Val Acc: 0.6866
Epoch 10/10, Loss: 0.2004, Acc: 0.9263, Val Loss: 1.0504, Val Acc: 0.6915


> The median of training accuracy for all the epochs for this model?

In [16]:
np.median(history1['acc'])

np.float64(0.8225)

> The standard deviation of training loss for all the epochs for this model?

In [17]:
np.std(history1['loss'])

np.float64(0.14849673501797045)

#### Data Augmentation

In [18]:
train_transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [19]:
train_data = datasets.ImageFolder(
    root="./data/train",
    transform=train_transform,
    target_transform=None
)

val_data = datasets.ImageFolder(
    root="./data/test",
    transform=val_transform
)

print(f"Train:\n{train_data}\nVal:\n{val_data}")

Train:
Dataset ImageFolder
    Number of datapoints: 800
    Root location: ./data/train
    StandardTransform
Transform: Compose(
               Resize(size=(200, 200), interpolation=bilinear, max_size=None, antialias=True)
               RandomRotation(degrees=[-50.0, 50.0], interpolation=nearest, expand=False, fill=0)
               RandomResizedCrop(size=(200, 200), scale=(0.9, 1.0), ratio=(0.9, 1.1), interpolation=bilinear, antialias=True)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
Val:
Dataset ImageFolder
    Number of datapoints: 201
    Root location: ./data/test
    StandardTransform
Transform: Compose(
               Resize(size=(200, 200), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )


In [20]:
train_dl = DataLoader(
    dataset=train_data,
    batch_size=20,
    shuffle=True
)

val_dl = DataLoader(
    dataset=val_data,
    batch_size=20,
    shuffle=False
)

train_dl, val_dl

(<torch.utils.data.dataloader.DataLoader at 0x7ab14d6fd370>,
 <torch.utils.data.dataloader.DataLoader at 0x7ab14d707860>)

In [21]:
num_epochs = 10
history2 = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_dl:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_data)
    epoch_acc = correct_train / total_train
    history2['loss'].append(epoch_loss)
    history2['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in val_dl:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(val_data)
    val_epoch_acc = correct_val / total_val
    history2['val_loss'].append(val_epoch_loss)
    history2['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6302, Acc: 0.6713, Val Loss: 0.6168, Val Acc: 0.6766
Epoch 2/10, Loss: 0.6037, Acc: 0.6575, Val Loss: 0.6981, Val Acc: 0.7015
Epoch 3/10, Loss: 0.5916, Acc: 0.6575, Val Loss: 0.6259, Val Acc: 0.6915
Epoch 4/10, Loss: 0.5697, Acc: 0.6787, Val Loss: 0.6705, Val Acc: 0.7015
Epoch 5/10, Loss: 0.5486, Acc: 0.6987, Val Loss: 0.7482, Val Acc: 0.6816
Epoch 6/10, Loss: 0.5508, Acc: 0.7125, Val Loss: 0.6131, Val Acc: 0.7264
Epoch 7/10, Loss: 0.5347, Acc: 0.7188, Val Loss: 0.5627, Val Acc: 0.7512
Epoch 8/10, Loss: 0.5105, Acc: 0.7300, Val Loss: 0.5742, Val Acc: 0.7214
Epoch 9/10, Loss: 0.5190, Acc: 0.7400, Val Loss: 0.6401, Val Acc: 0.6716
Epoch 10/10, Loss: 0.4902, Acc: 0.7562, Val Loss: 0.5761, Val Acc: 0.7214


> The mean of test loss for all the epochs for the model trained with augmentations?

In [22]:
np.mean(history2['val_loss'])

np.float64(0.6325862905872401)

> The average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?

In [23]:
np.mean(history2['val_acc'][5:])

np.float64(0.718407960199005)